In [1]:
import pandas as pd
import numpy as np
from date_continuity_check import pandas_inferred_frequency,frequency,is_continuous

In [2]:
#data folder path definition
data_path='c:\\Users\\seyma\\personal_work\\sanofi\\Sanofi_Challenge\\Data\\'

In [3]:
#first load the excel document
Excel_file=pd.ExcelFile(data_path+"Datainput.xlsx")

#create data frames for each sheet in the excel file
Combined_Yields=pd.read_excel(Excel_file,sheet_name="Combined_Yields")
QC_Data=pd.read_excel(Excel_file,sheet_name="QC_Data")
IoT=pd.read_excel(Excel_file,sheet_name="IoT")
Campaign_Info=pd.read_excel(Excel_file,sheet_name="Campaign_Info")
Batch_Genealogy=pd.read_excel(Excel_file,sheet_name="Batch_Genealogy")

In the Document, they said *"You can use the “activity_coefficient” (column YIELD_NAME) of these output batches as the target in your model"*


So, Let's begin with filtering materials who has YIELD_NAME equal to “activity_coefficient”

In [4]:
Combined_Yields[Combined_Yields.YIELD_NAME=="activity_coefficient"]

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,STAGE,YIELD_NAME,YIELD_VALUE
0,120932,P10446,20200612,s2,activity_coefficient,1.335753
1,120928,P10444,20200924,s2,activity_coefficient,1.275753
2,120935,P10444,20201015,s2,activity_coefficient,1.299432
3,120928,P10446,20200511,s2,activity_coefficient,1.324042
4,119136,P10344,20190329,s2,activity_coefficient,1.332979
...,...,...,...,...,...,...
168,122826,P10449,20221006,s2,activity_coefficient,1.124211
169,122810,P10148,20220831,s2,activity_coefficient,NaN
170,122828,P10449,20221028,s2,activity_coefficient,1.105905
171,122829,P10449,20221117,s2,activity_coefficient,1.134234


We have 173 data points but 65 of them have missing value on target variable.

In [5]:
Combined_Yields[Combined_Yields.YIELD_NAME=="activity_coefficient"].isna().sum()

BATCH_ID             0
MATERIAL_ID          0
MANUFACTURE_DATE     0
STAGE                0
YIELD_NAME           0
YIELD_VALUE         65
dtype: int64

I have decided to dropping those missing values. However, there may some ways to fill them but it is time consuming.

In [6]:
Combined_Yields_full=Combined_Yields[Combined_Yields.YIELD_NAME=="activity_coefficient"].dropna()

In [7]:
Combined_Yields_full.isna().sum()

BATCH_ID            0
MATERIAL_ID         0
MANUFACTURE_DATE    0
STAGE               0
YIELD_NAME          0
YIELD_VALUE         0
dtype: int64

In [8]:
#Transforming type from string to DateTime
Combined_Yields_full['MANUFACTURE_DATE'] = pd.to_datetime(Combined_Yields_full['MANUFACTURE_DATE'], format='%Y%m%d')

#for better understanding, Sorting values in ASC order
Combined_Yields_full = Combined_Yields_full.sort_values('MANUFACTURE_DATE')

I will drop STAGE and YIELD_NAME columns because they are consisting only one value. They are not differenciating.

In [9]:
Combined_Yields_full=Combined_Yields_full.drop(['STAGE','YIELD_NAME'], axis=1)

In [10]:
unique_combinations = Combined_Yields_full[['BATCH_ID', 'MATERIAL_ID','MANUFACTURE_DATE']].nunique()
print("# of unique: ")
print(unique_combinations)

# of unique: 
BATCH_ID             82
MATERIAL_ID           9
MANUFACTURE_DATE    106
dtype: int64


In [11]:
Combined_Yields_full

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,YIELD_VALUE
19,116785,P10344,2018-02-28,1.403015
20,116786,P10344,2018-03-23,1.368750
18,116787,P10344,2018-04-16,1.293267
24,116788,P10344,2018-06-14,1.326514
4,119136,P10344,2019-03-29,1.332979
...,...,...,...,...
153,122816,P10444,2022-12-01,1.522965
152,122818,P10444,2022-12-08,1.495911
158,122820,P10444,2022-12-15,1.496574
115,122821,P10444,2022-12-22,1.490813


I have 108 Output Batches. In the end Parent Batches will also join so, train data set may expand.

I will join __Campaign_Info table__ with __my Combined_Yields_full__ to use campain id and batch position

In [12]:
Campaign_Info

,BATCH_ID,MATERIAL_ID,CAMPAIGN_ID,BATCH_POSITION
0,119114.0,P10343,12.0,1.0
1,122803.0,P10450,62.0,1.0
2,121831.0,P10449,37.0,1.0
3,121832.0,P10449,37.0,2.0
4,122828.0,P10449,63.0,1.0
...,...,...,...,...
152,122817.0,P10449,48.0,2.0
153,119133.0,P10344,3.0,1.0
154,119134.0,P10344,3.0,2.0
155,119135.0,P10344,3.0,3.0


In [13]:
joined_df = pd.merge(Combined_Yields_full, Campaign_Info, on=['BATCH_ID', 'MATERIAL_ID'])

In [14]:
joined_df.isna().sum()

BATCH_ID            0
MATERIAL_ID         0
MANUFACTURE_DATE    0
YIELD_VALUE         0
CAMPAIGN_ID         7
BATCH_POSITION      7
dtype: int64

In [15]:
joined_df=joined_df.dropna()

In [16]:
joined_df.isna().sum()

BATCH_ID            0
MATERIAL_ID         0
MANUFACTURE_DATE    0
YIELD_VALUE         0
CAMPAIGN_ID         0
BATCH_POSITION      0
dtype: int64

In [17]:
joined_df

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,YIELD_VALUE,CAMPAIGN_ID,BATCH_POSITION
4,119136,P10344,2019-03-29,1.332979,3.0,4.0
5,119142,P10344,2019-05-29,1.247114,4.0,1.0
6,119144,P10344,2019-09-13,1.292268,5.0,1.0
7,119146,P10344,2019-11-05,1.329679,6.0,1.0
8,120933,P10349,2020-02-21,1.057945,18.0,1.0
...,...,...,...,...,...,...
102,122816,P10444,2022-12-01,1.522965,66.0,1.0
103,122818,P10444,2022-12-08,1.495911,66.0,2.0
104,122820,P10444,2022-12-15,1.496574,66.0,3.0
105,122821,P10444,2022-12-22,1.490813,66.0,4.0


The rows that don't exist in Campain_Info table dropped. So, data set shrinked from 108 to 100

So, we have IoT sheet, I need to join this table as well. It will be same join on material and batch ids

In [18]:
joined_df = pd.merge(joined_df, IoT, on=['BATCH_ID', 'MATERIAL_ID'])

In [19]:
column_na_counts = joined_df.isna().sum()

# Iterate over the column_na_counts Series and print each column along with its corresponding NaN count
for column, count in column_na_counts.items():
    print(f"{column}: {count}")

BATCH_ID: 0
MATERIAL_ID: 0
MANUFACTURE_DATE: 0
YIELD_VALUE: 0
CAMPAIGN_ID: 0
BATCH_POSITION: 0
campagne finance: 0
1_con_dur: 1
1_con_temp: 1
1_fil_dur: 1
1_final PH: 1
2_alka_hydro_dur: 1
2_fil_dur: 1
2_final_PH: 1
3_1a_etoh rto: 1
3_1a_etoh temp: 1
3_1a_hep concent: 1
3_1a_nacl cont: 1
3_1a_settl_dur: 1
3_2a_initial PH: 1
3_2a_etoh rto: 1
3_2a_hep cont: 1
3_2a_nacl cont: 1
3_2b_settl_dur: 0
3_2a_etoh temp: 1
3_2a_temp of hep solution: 1
4_dry extr_lod: 0
4_fil_dur: 0
4_Total vol: 0
4-2_etoh_rto: 0
4-3 etoh cont_Mother_liquor: 0
4-3_etoh cont: 0
stage5_drying_dur: 1
anti_iia_sur_sec: 0
3_2b_vol_red supernatant_UOL: 0
3_2b_vol precipitate: 0
3_2b_temp of hep solution: 0
3_2b_PH_before feeding: 0
3_2b_PH after feeding: 0
3_2b_hep concent: 0
3_2b_etoh_temp: 0
3_2b_etoh_rto: 0
3_2a_vol red_supernatant_UOL: 1
3_2a_vol precipitate: 1
3_2a_total vol after settl: 1
3_2a_settl_dur: 1
3_2a_nacl cont_: 1
3_2a_final PH: 1
3_1b_vol red supernatant_UOL: 23
3_1b_vol precipitate: 23
3_1b_total vol: 1

In [20]:
joined_df

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,YIELD_VALUE,CAMPAIGN_ID,BATCH_POSITION,campagne finance,1_con_dur,1_con_temp,1_fil_dur,...,3_2_settl_dur,3_2_temp of hep solution,3_2_vol precipitate,3_2_vol_red supernatant_UOL,stage_3_1b_nacl cont,3_2_vol reactor,3_2a_vol reactor,3_2b_vol reactor,fet_3_1a_final PH,fet_3_1a_initial PH
0,120930,P10449,2020-07-08,1.043462,29.0,1.0,29,77.0,81.0,544.0,...,731.5,19.0,200.0,0.066667,0.1024,5962.0,2974.0,2988.0,5.3,5.5
1,120932,P10449,2020-07-13,1.035355,29.0,2.0,29,78.0,81.0,1500.0,...,741.0,22.0,185.0,0.043333,0.1012,5970.0,2978.0,2992.0,5.3,6.1
2,120934,P10449,2020-07-21,1.056216,29.0,3.0,29,78.0,80.0,525.0,...,732.5,19.0,195.0,0.016667,0.1024,5955.0,2970.0,2985.0,5.3,5.4
3,120928,P10444,2020-09-24,1.275753,30.0,1.0,30,75.0,77.0,681.0,...,735.0,19.0,145.0,0.033333,0.0995,5972.0,2977.0,2995.0,5.3,5.4
4,120930,P10444,2020-10-01,1.395169,30.0,2.0,30,75.0,80.0,615.0,...,803.5,19.0,140.0,0.036667,0.1005,5968.0,2976.0,2992.0,5.3,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,122816,P10444,2022-12-01,1.522965,66.0,1.0,66,80.0,79.0,611.0,...,730.5,19.0,155.0,0.066667,0.1009,5978.0,2986.0,2992.0,5.3,5.4
87,122818,P10444,2022-12-08,1.495911,66.0,2.0,66,78.0,79.0,577.0,...,721.5,19.0,150.0,0.066667,0.1058,5969.0,2977.0,2992.0,5.3,5.3
88,122820,P10444,2022-12-15,1.496574,66.0,3.0,66,79.0,79.0,581.0,...,722.0,19.0,150.0,0.066667,0.0980,5967.0,2979.0,2988.0,5.3,5.4
89,122821,P10444,2022-12-22,1.490813,66.0,4.0,66,76.0,79.0,582.0,...,723.0,19.0,155.0,0.066667,0.0996,5967.0,2977.0,2990.0,5.3,5.4


One more join left. It is QC_Data

In [21]:
QC_Data

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,NEW_QC_TEST,VALUE
0,122875,P10051,20220704,pere_pour_azo,1.8
1,115705,P10005,20170721,cend_sul(fourni),NaN
2,115703,P10144,20171031,lla_fourni,NaN
3,120932,P10010,20200318,PH,6.9
4,122882,P10051,20220829,xa_tel_fourni,NaN
...,...,...,...,...,...
428973,122750,P10147,20201214,sul_ntu(fourni),NaN
428974,116258,P10044,20170109,cend_c1_creuset2,NaN
428975,118367,P10044,20180730,azot(fourni),NaN
428976,117224,P90111,20181127,than_fourni,NaN


In [22]:
#there were some zeros on the "MANUFACTURE_DATE" column. So, replaced them with NaN
QC_Data['MANUFACTURE_DATE'] = QC_Data['MANUFACTURE_DATE'].replace(0, np.nan)

#Transforming type from string to DateTime
QC_Data['MANUFACTURE_DATE'] = pd.to_datetime(QC_Data['MANUFACTURE_DATE'], format='%Y%m%d')

#for better understanding, Sorting values in ASC order
QC_Data = QC_Data.sort_values('MANUFACTURE_DATE')

In [23]:
QC_Data.isna().sum()

BATCH_ID                 0
MATERIAL_ID              0
MANUFACTURE_DATE      4070
NEW_QC_TEST              0
VALUE               372968
dtype: int64

Most of the test result is empty. So I want to see which test metrics have a value

In [24]:
QC_Data[QC_Data["VALUE"].isna()==False]

,BATCH_ID,MATERIAL_ID,MANUFACTURE_DATE,NEW_QC_TEST,VALUE
162442,115078,P10344,2014-05-02,PH,6.500000
65814,115078,P10344,2014-05-02,ratio_xa_lla(sec),0.995146
132530,115078,P10344,2014-05-02,lla_sec,207.870838
381411,115078,P10344,2014-05-02,absorbance,0.053000
116709,115078,P10344,2014-05-02,lla_tel,206.000000
...,...,...,...,...,...
424401,119113,P10003,NaT,PH,7.900000
425262,119113,P10003,NaT,lod,3.800000
425541,119124,P10003,NaT,perte,0.999600
425721,117496,P10005,NaT,coag_wet,175.000000


In [25]:
print("List of QC Tests that have value:")
print(QC_Data[QC_Data["VALUE"].isna()==False]["NEW_QC_TEST"].unique())
print("----------------------------------------------------------------")
print("Length of list:")
print(len(QC_Data[QC_Data["VALUE"].isna()==False]["NEW_QC_TEST"].unique()))

List of QC Tests that have value:
['PH' 'ratio_xa_lla(sec)' 'lla_sec' 'absorbance' 'lla_tel' 'xa_dry'
 'sul_ntu' 'xa_tel' 'lod' 'adn' 'coag_wet' 'coag_dry' 'anticoagulante '
 'masse' 'moyenne' 'ratio_80_24' 'teneur_eau' 'proteiniq' 'azot_sec'
 'otdlq_260' 'cend_sul' 'ethan' 'rende_la' 'otidiq' 'cal_sec' 'lines_sec'
 'otidiq_hplc' 'sod_sec' 'rende_uimg' 'perte' 'pertem2' 'pertem1'
 'robie_to' 'es2_perte' 'perem1cru1' 'perte_cre2' 'dessic_creuset'
 'dessic_moyenne' 'pertem1cru2' 'pertem2_cre1' 'pertem2_cre2' 'perte_cre1'
 'thanol' 'teneur_chon' 'butan' 'inque_lowry' 'pere_pour_lla'
 'cend_sul_creuset1' 'pere_pour_azo' 'es1_azo_sur_sec_pct'
 'cend_m_creuset1' 'es1_azo_sur_tel' 'cend_c2_creuset2' 'cend_sul_moy'
 'es2_azo_sur_tel' 'cend_sul_creuset2' 'cend_m_creuset2'
 'es2_azo_sur_sec_pct' 'cend_c2_creuset1' 'cend_c1_creuset2'
 'cend_c1_creuset1' 'ratio_xa_llafourni' 'than_fourni' 'xa_tel_fourni'
 'lla_fourni' 'xa_sec_fourni' 'lla_tel_fourni' 'enzym' 'azot(fourni)'
 'ratio_xa_lla(fourni)' 

In [26]:
print(QC_Data["MANUFACTURE_DATE"].max())
print(QC_Data["MANUFACTURE_DATE"].min())

2023-01-16 00:00:00
2014-05-02 00:00:00


In [27]:
unique_combinations = QC_Data[['BATCH_ID', 'MATERIAL_ID','NEW_QC_TEST','MANUFACTURE_DATE']].nunique()
print("# of unique: ")
print(unique_combinations)

# of unique: 
BATCH_ID            5181
MATERIAL_ID           40
NEW_QC_TEST           74
MANUFACTURE_DATE    1223
dtype: int64


In [28]:
#this is for understanding how many different pairs we have
unique_pairs_count = QC_Data[['BATCH_ID', 'MATERIAL_ID']].drop_duplicates().shape[0]
print(unique_pairs_count)


5797


In [29]:
#joined_df = pd.merge(joined_df, QC_Data, on=['BATCH_ID', 'MATERIAL_ID'])

Batch Genealogy

In [30]:
Batch_Genealogy

,BATCH_ID,MATERIAL_ID,LEVEL,TREE_ID,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION
0,122812,P10449,2,02d31e7f93f5de3cb5b7fcc8ab00426f4f57f7fa33dad5...,R104490026,20220203,232.100,KG,101,hep_so
1,122813,P10449,2,02d31e7f93f5de3cb5b7fcc8ab00426f4f57f7fa33dad5...,R104490027,20220211,253.550,KG,101,hep_so
2,122825,P10449,2,0347d96c7e652f3ac24b2bf917652763581f0dc1b62594...,R104490036,20220916,265.100,KG,101,hep_so
3,121843,P10449,2,035e3927189e6c90047992537b968bde33f46a3b1acb9d...,R104490023,20211105,223.900,KG,101,hep_so
4,121823,P10450,2,035e3927189e6c90047992537b968bde33f46a3b1acb9d...,R104500007,20211021,227.200,KG,101,hep_so
...,...,...,...,...,...,...,...,...,...,...
4192,121907,P10010,2,4f6560e1695aacd541ba25c5ee8bf7ba7b013176c04354...,R104500005,20210106,25.035,KG,261,hep_bru_dc
4193,121908,P10010,2,243b6ba66876069e071084a9ba262c682c42b37e615588...,R104500005,20210113,26.041,KG,261,hep_bru_dc
4194,121908,P10010,2,4f6560e1695aacd541ba25c5ee8bf7ba7b013176c04354...,R104500005,20210113,26.041,KG,261,hep_bru_dc
4195,121909,P10010,2,243b6ba66876069e071084a9ba262c682c42b37e615588...,R104500005,20210114,5.916,KG,261,hep_bru_dc


In [31]:
Batch_Genealogy["MOVEMENT_TYPE"].unique()

array([101, 531, 261], dtype=int64)

In [32]:
Batch_Genealogy["LEVEL"].unique()

array([2], dtype=int64)

We have 3 different types of movement. 531 can be disregarded according to document. Also, Level column is not necessary because it stores only one variable for each row.

In [33]:
# Delete rows where MOVEMENT_TYPE is equal to 531
Batch_Genealogy = Batch_Genealogy[Batch_Genealogy['MOVEMENT_TYPE'] != 531]

# Remove the LEVEL column
Batch_Genealogy = Batch_Genealogy.drop('LEVEL', axis=1)

In [34]:
Batch_Genealogy["MOVEMENT_TYPE"].unique()

array([101, 261], dtype=int64)

In [35]:
#there were some zeros on the "MANUFACTURE_DATE" column. So, replaced them with NaN
Batch_Genealogy['MANUFACTURE_DATE'] = Batch_Genealogy['MANUFACTURE_DATE'].replace(0, np.nan)

#Transforming type from string to DateTime
Batch_Genealogy['MANUFACTURE_DATE'] = pd.to_datetime(Batch_Genealogy['MANUFACTURE_DATE'], format='%Y%m%d')

#for better understanding, Sorting values in ASC order
Batch_Genealogy = Batch_Genealogy.sort_values('MANUFACTURE_DATE')

In [39]:
#this is for understanding how many different pairs we have
unique_pairs_count = Batch_Genealogy[['BATCH_ID', 'MATERIAL_ID']].drop_duplicates().shape[0]
print(unique_pairs_count)


983


In [40]:
Batch_Genealogy.isna().sum()

BATCH_ID                     0
MATERIAL_ID                  0
TREE_ID                      0
PROCESS_ORDER_NUMBER         0
MANUFACTURE_DATE            38
QUANTITY                     0
UNIT_OF_MEASURE              0
MOVEMENT_TYPE                0
NEW_MATERIAL_DESCRIPTION     0
dtype: int64

In [41]:
Batch_Genealogy

,BATCH_ID,MATERIAL_ID,TREE_ID,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION
705,115685,P10004,810e9cc06b5b83951e947a08130027227bd7a69446bdaa...,R104440001,2017-02-09,0.900,KG,261,hep_bru_li
694,115685,P10004,6a3f7ab370155694595e55d5a9ffca42b864dae2faa26e...,R104440001,2017-02-09,0.900,KG,261,hep_bru_li
770,115693,P10004,810e9cc06b5b83951e947a08130027227bd7a69446bdaa...,R104440001,2017-05-22,0.850,KG,261,hep_bru_li
759,115693,P10004,6a3f7ab370155694595e55d5a9ffca42b864dae2faa26e...,R104440001,2017-05-22,0.850,KG,261,hep_bru_li
636,117268,P10004,810e9cc06b5b83951e947a08130027227bd7a69446bdaa...,R104440005,2018-04-04,0.900,KG,261,hep_bru_li
...,...,...,...,...,...,...,...,...,...
2830,121581,P10005,87a29ab221a65ffec959eb711ae30461322fe83d42e9fb...,R104490010,NaT,25.476,KG,261,hep_bru_ck
2863,121581,P10005,bd255421b1a6fae818c27cb5f5c9355b276eaaf9454b41...,R104490010,NaT,25.476,KG,261,hep_bru_ck
3854,112924,P20098,2e058cc192750ec5c430545d046041b4dcf7dcccc0d117...,R104490001,NaT,2.000,1,261,fil22
3876,112924,P20098,a0754ba875ba2443c3c091d2069be5e392a4708d0ace61...,R104490001,NaT,2.000,1,261,fil22


In [42]:
unique_combinations = Batch_Genealogy[['BATCH_ID', 'MATERIAL_ID','TREE_ID','PROCESS_ORDER_NUMBER','MANUFACTURE_DATE','MOVEMENT_TYPE']].nunique()
print("# of unique: ")
print(unique_combinations)

# of unique: 
BATCH_ID                931
MATERIAL_ID              20
TREE_ID                 137
PROCESS_ORDER_NUMBER     97
MANUFACTURE_DATE        477
MOVEMENT_TYPE             2
dtype: int64


In [69]:
#output batches
output_batches=Batch_Genealogy[Batch_Genealogy["MOVEMENT_TYPE"]==101]
output_batches

,BATCH_ID,MATERIAL_ID,TREE_ID,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION
38,120928,P10446,32165b2eddcfb26b0a0a1409bf0d1c3e5230c42fdd426f...,R104460001,2020-05-11,165.62,KG,101,hep_so
62,120928,P10446,4ea6a53354958e942d9b5f9b498b197a09490b79c4cfe2...,R104460001,2020-05-11,165.62,KG,101,hep_so
63,120930,P10446,4ea6a53354958e942d9b5f9b498b197a09490b79c4cfe2...,R104460002,2020-05-14,209.63,KG,101,hep_so
161,120930,P10446,be5eca36c7f1eb270223aa08a378e34d808cf25f7b7ce2...,R104460002,2020-05-14,209.63,KG,101,hep_so
79,120932,P10446,6a3f7ab370155694595e55d5a9ffca42b864dae2faa26e...,R104460003,2020-06-12,182.80,KG,101,hep_so
...,...,...,...,...,...,...,...,...,...
45,122820,P10444,385afbeede879098c49e04e818e057cb755ce5d27f0830...,R104440031,2022-12-15,187.60,KG,101,hep_so
113,122820,P10444,88c6170522dad51ef9fddbc564d3da83a7d484184981be...,R104440031,2022-12-15,187.60,KG,101,hep_so
92,122821,P10444,79338195ac21acb995cbd61803fd4d5045f892ac51f6c8...,R104440032,2022-12-22,205.55,KG,101,hep_so
46,122821,P10444,385afbeede879098c49e04e818e057cb755ce5d27f0830...,R104440032,2022-12-22,205.55,KG,101,hep_so


In [71]:
#parent batches
parent_batches=Batch_Genealogy[Batch_Genealogy["MOVEMENT_TYPE"]==261]
parent_batches

,BATCH_ID,MATERIAL_ID,TREE_ID,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION
705,115685,P10004,810e9cc06b5b83951e947a08130027227bd7a69446bdaa...,R104440001,2017-02-09,0.900,KG,261,hep_bru_li
694,115685,P10004,6a3f7ab370155694595e55d5a9ffca42b864dae2faa26e...,R104440001,2017-02-09,0.900,KG,261,hep_bru_li
770,115693,P10004,810e9cc06b5b83951e947a08130027227bd7a69446bdaa...,R104440001,2017-05-22,0.850,KG,261,hep_bru_li
759,115693,P10004,6a3f7ab370155694595e55d5a9ffca42b864dae2faa26e...,R104440001,2017-05-22,0.850,KG,261,hep_bru_li
636,117268,P10004,810e9cc06b5b83951e947a08130027227bd7a69446bdaa...,R104440005,2018-04-04,0.900,KG,261,hep_bru_li
...,...,...,...,...,...,...,...,...,...
2830,121581,P10005,87a29ab221a65ffec959eb711ae30461322fe83d42e9fb...,R104490010,NaT,25.476,KG,261,hep_bru_ck
2863,121581,P10005,bd255421b1a6fae818c27cb5f5c9355b276eaaf9454b41...,R104490010,NaT,25.476,KG,261,hep_bru_ck
3854,112924,P20098,2e058cc192750ec5c430545d046041b4dcf7dcccc0d117...,R104490001,NaT,2.000,1,261,fil22
3876,112924,P20098,a0754ba875ba2443c3c091d2069be5e392a4708d0ace61...,R104490001,NaT,2.000,1,261,fil22


In [70]:
print("OUTPUT BATCHES")
print("-------------------------------")
unique_combinations = output_batches[['BATCH_ID', 'MATERIAL_ID','TREE_ID','PROCESS_ORDER_NUMBER','MANUFACTURE_DATE','MOVEMENT_TYPE']].nunique()
print("Length of df: {}".format(len(output_batches)))
print("# of unique: ")
print(unique_combinations)

OUTPUT BATCHES
-------------------------------
Length of df: 208
# of unique: 
BATCH_ID                 74
MATERIAL_ID               5
TREE_ID                 137
PROCESS_ORDER_NUMBER     97
MANUFACTURE_DATE         95
MOVEMENT_TYPE             1
dtype: int64


In [72]:
print("PARENT BATCHES")
print("-------------------------------")
unique_combinations = parent_batches[['BATCH_ID', 'MATERIAL_ID','TREE_ID','PROCESS_ORDER_NUMBER','MANUFACTURE_DATE','MOVEMENT_TYPE']].nunique()
print("Length of df: {}".format(len(parent_batches)))
print("# of unique: ")
print(unique_combinations)

PARENT BATCHES
-------------------------------
Length of df: 3781
# of unique: 
BATCH_ID                885
MATERIAL_ID              16
TREE_ID                 137
PROCESS_ORDER_NUMBER     97
MANUFACTURE_DATE        439
MOVEMENT_TYPE             1
dtype: int64


This is underlies the statement in the document: *iv.	Output batches, which have movement type "101"  in the Batch_Genealogy sheet, are the ones used in yield calculation – they should have the target yield numbers in sheet Combined_Yields*

In [73]:
common_ids = set(output_batches['BATCH_ID']).intersection(set(Combined_Yields_full['BATCH_ID']))
print(len(common_ids))

73


In [74]:
common_ids = set(output_batches['MATERIAL_ID']).intersection(set(parent_batches['MATERIAL_ID']))
print(common_ids)

{'P10444'}


In [75]:
Batch_Genealogy[(Batch_Genealogy["MATERIAL_ID"]=="P10444")&(Batch_Genealogy["MOVEMENT_TYPE"]==261)]

,BATCH_ID,MATERIAL_ID,TREE_ID,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION
211,120939,P10444,45ae0fa98b910d382ecc91f707eb2aca0237354dc0f2fc...,R104440009R,2020-12-14,178.55,KG,261,hep_so
210,120939,P10444,3e817432abfaad60476e1898ad30732a1f8cddf91aa90b...,R104440009R,2020-12-14,178.55,KG,261,hep_so


In [108]:
#one tree_id
Batch_Genealogy[(Batch_Genealogy["PROCESS_ORDER_NUMBER"]=="R104440013")]

,BATCH_ID,MATERIAL_ID,TREE_ID,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION
282,113513,P20098,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-04-27,2.0,1,261,fil22
283,113526,P20097,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-05-05,4.0,1,261,fil45
293,113549,P20094,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-05-28,48.3,L,261,sod3n
339,113558,P20095,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-06-03,33.0,KG,261,eauoxy
414,113575,P20003,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-06-09,24.0,KG,261,potasi
484,113610,P20002,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-07-21,30.0,KG,261,clarcel28
682,121330,P10004,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-09-29,300.0,KG,261,hep_bru_li
547,113741,P20093,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-11-06,26.1,L,261,acide_chlor
613,113743,P20096,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2020-11-06,687.5,KG,261,chlor_cal
42,120944,P10444,3475b6baa9edcedb02f27388c4312b27040cafaac0eae2...,R104440013,2021-01-18,164.3,KG,101,hep_so


In [111]:
#three tree_ids
Batch_Genealogy[(Batch_Genealogy["PROCESS_ORDER_NUMBER"]=="R104500018")]

,BATCH_ID,MATERIAL_ID,TREE_ID,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION
1673,114256,P20095,83de933a762cd4bac2c5d935dbb448d698bb24c65d4671...,R104500018,2021-07-12,32.89,KG,261,eauoxy
1647,114256,P20095,46e27ad1ede70dc2a2572418adfe770c448ed33bcf7831...,R104500018,2021-07-12,32.89,KG,261,eauoxy
1651,114256,P20095,4acc8753bf3001bb01758aa4f022c794a4c28f93240669...,R104500018,2021-07-12,32.89,KG,261,eauoxy
1770,114426,P20094,4acc8753bf3001bb01758aa4f022c794a4c28f93240669...,R104500018,2021-10-26,36.40,L,261,sod3n
1766,114426,P20094,46e27ad1ede70dc2a2572418adfe770c448ed33bcf7831...,R104500018,2021-10-26,36.40,L,261,sod3n
...,...,...,...,...,...,...,...,...,...
2694,115010,P20096,83de933a762cd4bac2c5d935dbb448d698bb24c65d4671...,R104500018,2022-10-13,312.60,KG,261,chlor_cal
2664,115010,P20096,46e27ad1ede70dc2a2572418adfe770c448ed33bcf7831...,R104500018,2022-10-13,312.60,KG,261,chlor_cal
52,122804,P10450,46e27ad1ede70dc2a2572418adfe770c448ed33bcf7831...,R104500018,2022-11-25,244.65,KG,101,hep_so
57,122804,P10450,4acc8753bf3001bb01758aa4f022c794a4c28f93240669...,R104500018,2022-11-25,244.65,KG,101,hep_so


In [84]:
len(Batch_Genealogy[(Batch_Genealogy["PROCESS_ORDER_NUMBER"]=="R104440005")]["MATERIAL_ID"].unique())

10

In [100]:
duplicate_tree_ids = Batch_Genealogy.groupby('PROCESS_ORDER_NUMBER')['TREE_ID'].unique()

print(duplicate_tree_ids)
duplicate_tree_ids.to_excel("C:\\Users\\seyma\\Downloads\\output_2.xlsx")

PROCESS_ORDER_NUMBER
R104430001    [9d559718f52d5c6ea7e4ac97ee8d8a25403c5846337e2...
R104430002    [79338195ac21acb995cbd61803fd4d5045f892ac51f6c...
R104440001    [810e9cc06b5b83951e947a08130027227bd7a69446bda...
R104440002    [aa19de414ad4a0e5c2572c5b819e63c9570e0944a888b...
R104440003    [5c24eb0b93bd3471e61ce16bef973f4b5b8792d9a0658...
                                    ...                        
R104500014    [8cd8f874c0baaab9088b1d89b4cb919c5643a8e85ead2...
R104500015    [8cd8f874c0baaab9088b1d89b4cb919c5643a8e85ead2...
R104500016    [c356221162346e56335d57edb48fcf2647211daaeb6bf...
R104500017    [c61162dadeefb402ac76434d767e8bf4ae7d126be2b96...
R104500018    [83de933a762cd4bac2c5d935dbb448d698bb24c65d467...
Name: TREE_ID, Length: 97, dtype: object
